In [2]:
import torch.nn as nn
import torch

# Create a 3D convolutional layer
# in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 1
conv_layer = nn.Conv3d(1, 32, 3, stride=1, padding=1)

# Example input tensor: (batch_size, channels, depth, height, width)
# For a single sample with 1 channel and 20x20x20 size
input_tensor = torch.randn(1, 1, 20, 20, 20)

# Apply the convolutional layer to the input
output_tensor = conv_layer(input_tensor)

print(output_tensor.shape)


torch.Size([1, 32, 20, 20, 20])


In [4]:
import nibabel as nib

# Load the NIfTI file
nifti_file = nib.load(r'data\first_fusion\002_S_0295_fused.nii')

# Get the data from the file
data = nifti_file.get_fdata()
print(data.shape)


(193, 229, 193)


In [5]:
# Print the affine matrix
print("Affine matrix:")
print(nifti_file.affine)

# Header might also contain useful orientation and dimension info
print("Header data:")
print(nifti_file.header)

Affine matrix:
[[   1.    0.    0.  -96.]
 [   0.    1.    0. -132.]
 [   0.    0.    1.  -74.]
 [   0.    0.    0.    1.]]
Header data:
<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b''
dim_info        : 0
dim             : [  3 193 229 193   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float64
bitpix          : 64
slice_start     : 0
pixdim          : [1. 1. 1. 1. 1. 1. 1. 1.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 0
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : unknown
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 0.0
quater

In [22]:
import torch
import torch.nn as nn

# Define the input tensor for a single grayscale 3D image
input_tensor = torch.randn(1, 1, 193, 229, 193)  # (batch_size, channels, depth, height, width)

# First convolutional layer
conv_layer1 = nn.Conv3d(in_channels=1, out_channels=32, kernel_size=64, stride=1, padding=1)
output1 = conv_layer1(input_tensor)
print("Output shape after first layer:", output1.shape)

# Second convolutional layer
conv_layer2 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=3, stride=2, padding=1)
output2 = conv_layer2(output1)
print("Output shape after second layer:", output2.shape)


Output shape after first layer: torch.Size([1, 32, 132, 168, 132])
Output shape after second layer: torch.Size([1, 64, 66, 84, 66])


In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN3D(nn.Module):
    def __init__(self):
        super(CNN3D, self).__init__()
        # Define the convolutional layers and pooling layer
        self.conv1 = nn.Conv3d(in_channels=1, out_channels=16, kernel_size=16, stride=4, padding=0)
        self.pool = nn.MaxPool3d(kernel_size=1)
        self.conv2 = nn.Conv3d(in_channels=16, out_channels=32, kernel_size=8, stride=2, padding=0)
        self.conv3 = nn.Conv3d(in_channels=32, out_channels=64, kernel_size=4, stride=2, padding=0)
        self.conv4 = nn.Conv3d(in_channels=64, out_channels=128, kernel_size=2, stride=2, padding=0)
        # Assuming some volume remains here for the sake of example, often requires actual calculation
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=128 * 128, out_features=1024)  # Placeholder for in_features
        self.fc2 = nn.Linear(in_features=1024, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)

    def forward(self, x):
        x = self.conv1(x)
        print("After Conv1:", x.shape)
        x = self.pool(x)
        print("After Pool1:", x.shape)
        
        x = self.conv2(x)
        print("After Conv2:", x.shape)
        x = self.pool(x)
        print("After Pool2:", x.shape)
        
        x = self.conv3(x)
        print("After Conv3:", x.shape)
        x = self.pool(x)
        print("After Pool3:", x.shape)
        
        x = self.conv4(x)
        print("After Conv4:", x.shape)
        x = self.pool(x)
        print("After Pool4:", x.shape)
        
        x = self.flatten(x)
        print("After Flatten:", x.shape)
        
        x = self.fc1(x)
        print("After FC1:", x.shape)
        
        x = F.relu(self.fc2(x))
        print("After FC2:", x.shape)
        
        x = F.relu(self.fc3(x))
        print("After FC3:", x.shape)
        
        x = torch.sigmoid(self.output(x))
        print("After Sigmoid:", x.shape)
        return x


In [24]:

# Create the model
model = CNN3D()

# Initialize a tensor to match the input size described
input_tensor = torch.randn(1, 1, 193, 229, 193)

# Pass the tensor through the model
output = model(input_tensor)

After Conv1: torch.Size([1, 16, 45, 54, 45])
After Pool1: torch.Size([1, 16, 45, 54, 45])
After Conv2: torch.Size([1, 32, 19, 24, 19])
After Pool2: torch.Size([1, 32, 19, 24, 19])
After Conv3: torch.Size([1, 64, 8, 11, 8])
After Pool3: torch.Size([1, 64, 8, 11, 8])
After Conv4: torch.Size([1, 128, 4, 5, 4])
After Pool4: torch.Size([1, 128, 4, 5, 4])
After Flatten: torch.Size([1, 10240])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10240 and 16384x1024)